In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

# Data preprocessing
train_path = '/content/drive/MyDrive/project file/archive (6)/Training'
test_path = '/content/drive/MyDrive/project file/archive (6)/Testing'

def load_data(data_path):
    data = []
    labels = []
    class_names = sorted(os.listdir(data_path))
    for class_index, class_name in enumerate(class_names):
        class_dir = os.path.join(data_path, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (64, 64))
            data.append(img)
            labels.append(class_index)
    return np.array(data), np.array(labels)

X_train, y_train = load_data(train_path)
X_test, y_test = load_data(test_path)

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [ ]:
def create_model(learning_rate, num_filters, dense_units, dropout_rate):
    model = Sequential()
    model.add(Conv2D(int(num_filters), (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(int(dense_units), activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:

def fitness_function(params):
    learning_rate, num_filters, dense_units, dropout_rate, batch_size = params

    model = create_model(learning_rate, num_filters, dense_units, dropout_rate)

    history = model.fit(
        X_train, y_train,
        epochs=5,
        batch_size=int(batch_size),
        validation_data=(X_test, y_test),
        verbose=0
    )

    val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)

    return val_loss


In [ ]:
# Define the GreyWolfOptimizer class
class GreyWolfOptimizer:
    def __init__(self, fitness_function, num_wolves=5, num_iterations=20, dim=5, lb=[0.0001, 16, 16, 0.1, 16], ub=[0.1, 128, 256, 0.5, 128]):
        self.fitness_function = fitness_function
        self.num_wolves = num_wolves
        self.num_iterations = num_iterations
        self.dim = dim
        self.lb = np.array(lb)
        self.ub = np.array(ub)
        self.alpha_pos = np.zeros(dim)
        self.alpha_score = float("inf")
        self.beta_pos = np.zeros(dim)
        self.beta_score = float("inf")
        self.delta_pos = np.zeros(dim)
        self.delta_score = float("inf")
        self.wolves = np.random.uniform(0, 1, (num_wolves, dim)) * (self.ub - self.lb) + self.lb

    def optimize(self):
        for t in range(self.num_iterations):
            for i in range(self.num_wolves):
                fitness = self.fitness_function(self.wolves[i])
                if fitness < self.alpha_score:
                    self.alpha_score = fitness
                    self.alpha_pos = self.wolves[i].copy()
                elif fitness < self.beta_score:
                    self.beta_score = fitness
                    self.beta_pos = self.wolves[i].copy()
                elif fitness < self.delta_score:
                    self.delta_score = fitness
                    self.delta_pos = self.wolves[i].copy()

            a = 2 - t * (2 / self.num_iterations)
            for i in range(self.num_wolves):
                for j in range(self.dim):
                    r1, r2 = np.random.random(), np.random.random()
                    A1, C1 = 2 * a * r1 - a, 2 * r2
                    D_alpha = abs(C1 * self.alpha_pos[j] - self.wolves[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha

                    r1, r2 = np.random.random(), np.random.random()
                    A2, C2 = 2 * a * r1 - a, 2 * r2
                    D_beta = abs(C2 * self.beta_pos[j] - self.wolves[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta

                    r1, r2 = np.random.random(), np.random.random()
                    A3, C3 = 2 * a * r1 - a, 2 * r2
                    D_delta = abs(C3 * self.delta_pos[j] - self.wolves[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta

                    self.wolves[i, j] = np.clip((X1 + X2 + X3) / 3, self.lb[j], self.ub[j])

        return self.alpha_pos, self.alpha_score

In [ ]:
# Initialize and run the optimizer
gwo = GreyWolfOptimizer(fitness_function, num_wolves=5, num_iterations=10, dim=5)
best_params, best_score = gwo.optimize()

# Extract the best parameters
best_learning_rate, best_num_filters, best_dense_units, best_dropout_rate, best_batch_size = best_params

print("Best Hyperparameters found by GWO:")
print("Learning Rate:", best_learning_rate)
print("Number of Filters:", best_num_filters)
print("Dense Units:", best_dense_units)
print("Dropout Rate:", best_dropout_rate)
print("Batch Size:", best_batch_size)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best Hyperparameters found by GWO:
Learning Rate: 0.004331728454545964
Number of Filters: 55.682031373748764
Dense Units: 84.6380624944274
Dropout Rate: 0.19905197328726285
Batch Size: 70.55672539484824


In [ ]:

# Train and evaluate using the best hyperparameters
model = create_model(best_learning_rate, best_num_filters, best_dense_units, best_dropout_rate)
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=int(best_batch_size),
    validation_data=(X_test, y_test),
    verbose=1
)

# Evaluate the model
final_loss, final_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Final Test Loss:", final_loss)
print("Final Test Accuracy:", final_accuracy)

Epoch 1/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.5395 - loss: 1.9613 - val_accuracy: 0.7544 - val_loss: 0.6700
Epoch 2/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8645 - loss: 0.3514 - val_accuracy: 0.8879 - val_loss: 0.3191
Epoch 3/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9386 - loss: 0.1733 - val_accuracy: 0.9047 - val_loss: 0.2561
Epoch 4/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9699 - loss: 0.0902 - val_accuracy: 0.9222 - val_loss: 0.2607
Epoch 5/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9782 - loss: 0.0628 - val_accuracy: 0.9367 - val_loss: 0.1987
Epoch 6/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9907 - loss: 0.0323 - val_accuracy: 0.9436 - val_loss: 0.1834
Epoch 7/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9963 - loss: 0.0187 - val_accuracy: 0.9336 - val_loss: 0.2293
Epoch 8/50
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9957 - loss: 0.0210 - val_accuracy: 0.9527 - val_lo

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

class_names = ['glioma', 'meningioma', 'no tumor', 'pituitary']

def preprocess_image(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    # Resize the image to match the input size of your model
    img = cv2.resize(img, (64, 64))  # Adjust size according to your model's input shape
    # Preprocess the image: normalize pixel values
    img = img.astype('float32') / 255.0
    # Expand dimensions to match the batch size used by your model
    img = np.expand_dims(img, axis=0)
    return img

def predict_tumor_type(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    # Predict tumor type using your model
    predictions = model.predict(img)
    # Get the predicted class index
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    # Map the index to the class name
    predicted_class = class_names[predicted_class_index]
    return predicted_class


image_path = '/content/drive/MyDrive/project file/archive (6)/Testing/notumor/Te-no_0013.jpg'
predicted_tumor_type = predict_tumor_type(image_path)
print('Predicted tumor type:', predicted_tumor_type)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
Predicted tumor type: no tumor


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

class_names = ['glioma', 'meningioma', 'no tumor', 'pituitary']

def preprocess_image(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    # Resize the image to match the input size of your model
    img = cv2.resize(img, (64, 64))
    # Preprocess the image: normalize pixel values
    img = img.astype('float32') / 255.0
    # Expand dimensions to match the batch size used by your model
    img = np.expand_dims(img, axis=0)
    return img

def predict_tumor_type(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    # Predict tumor type using your model
    predictions = model.predict(img)
    # Get the predicted class index
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    # Map the index to the class name
    predicted_class = class_names[predicted_class_index]
    return predicted_class


image_path = '/content/drive/MyDrive/project file/archive (6)/Testing/meningioma/Te-meTr_0004.jpg'
predicted_tumor_type = predict_tumor_type(image_path)
print('Predicted tumor type:', predicted_tumor_type)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted tumor type: meningioma
